In [29]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [30]:
import sys
sys.path.append('/content/drive/MyDrive/GAN Stuff/models')
sys.path.append('/content/drive/MyDrive/GAN Stuff/functions')

In [31]:
import numpy as np
import torch
from torch import nn
from torchvision.models import inception_v3
from torch.nn.functional import softmax

from WGAN import *
from VAE import *
from data_functions import *
from models_functions import *

In [ ]:
%cp -av /content/drive/MyDrive/'Giraffa Analytics'/'MRI-Blur-Detection'/'Data' 'preprocessed slices'

'/content/drive/MyDrive/Giraffa Analytics/MRI-Blur-Detection/Data' -> 'preprocessed slices'
'/content/drive/MyDrive/Giraffa Analytics/MRI-Blur-Detection/Data/preprocessed slices' -> 'preprocessed slices/preprocessed slices'
'/content/drive/MyDrive/Giraffa Analytics/MRI-Blur-Detection/Data/preprocessed slices/slice_14_NC247_nomotion.npy' -> 'preprocessed slices/preprocessed slices/slice_14_NC247_nomotion.npy'
'/content/drive/MyDrive/Giraffa Analytics/MRI-Blur-Detection/Data/preprocessed slices/slice_14_NC248_motion.npy' -> 'preprocessed slices/preprocessed slices/slice_14_NC248_motion.npy'
'/content/drive/MyDrive/Giraffa Analytics/MRI-Blur-Detection/Data/preprocessed slices/slice_14_NC248_nomotion.npy' -> 'preprocessed slices/preprocessed slices/slice_14_NC248_nomotion.npy'
'/content/drive/MyDrive/Giraffa Analytics/MRI-Blur-Detection/Data/preprocessed slices/slice_14_NC249_motion.npy' -> 'preprocessed slices/preprocessed slices/slice_14_NC249_motion.npy'
'/content/drive/MyDrive/Giraffa 

In [ ]:
img_shape = (1, 126, 126)

images = get_images(img_shape=img_shape)

dataset = CustomDataset(images)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [32]:
z_dim = 128
device = 'cpu'
generator = Generator(z_dim).to(device)
generator.load_state_dict(torch.load('drive/MyDrive/GAN Stuff/generator.pth', map_location=torch.device(device)))
generator.eval()

Generator(
  (gen): Sequential(
    (0): Sequential(
      (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): ConvTranspose2d(64, 256, kernel_size=(4, 4), stride=(3, 3))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): ConvTranspose2d(256, 128, kernel_size=(5, 5), stride=(2, 2))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (3): Sequential(
      (0): ConvTranspose2d(128, 64, kernel_size=(5, 5), stride=(2, 2))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (4): Sequential(
      (0): ConvTranspose2d(64, 1, kernel_size=(6, 6), stride=(2, 2))

In [33]:
critic = Critic().to(device)
critic.load_state_dict(torch.load('drive/MyDrive/GAN Stuff/crit.pth', map_location=torch.device(device)))
critic.eval()

Critic(
  (crit): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (2): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (3): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2))
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (4): Sequential(
      

In [ ]:
import torch
from torch import nn
from torchvision.models import inception_v3
from torch.nn.functional import softmax
from sklearn.model_selection import KFold

# Initialize the InceptionV3 model outside the functions
try:
  inception_model = inception_v3(pretrained=True, transform_input=False).cuda()
except:
  inception_model = inception_v3(pretrained=True, transform_input=False)
inception_model.eval()

# Define a function to repeat the single channel to create a pseudo three-channel image
def repeat_channel(image):
    # Repeat the single channel to create a pseudo three-channel image
    return image.repeat(1, 3, 1, 1)

# Define a function to calculate the Inception Score with k-fold cross-validation for WGAN
def calculate_inception_score_cv_wgan(generator, get_noise, num_samples, k_folds=5, device='cuda'):
    # Set the generator to evaluation mode
    generator.eval()

    # # Generate samples using the generator
    with torch.inference_mode():
      noise = get_noise(num_samples, z_dim, device=device)
      all_samples = generator(noise)

    # Create a list to store Inception Scores for each fold
    inception_scores_fake = []

    # Perform k-fold cross-validation
    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    for _, test_idx in kf.split(all_samples):
        # Create data loaders for the current fold
        test_sampler = torch.utils.data.SubsetRandomSampler(test_idx)
        test_dataloader = torch.utils.data.DataLoader(all_samples, batch_size=64, sampler=test_sampler)

        # Compute the Inception Score for the current fold
        inception_score_fake = calculate_fold_inception_score_wgan(generator, get_noise, test_dataloader, device)
        inception_scores_fake.append(inception_score_fake)

    return inception_scores_fake

def calculate_fold_inception_score_wgan(generator, get_noise, test_dataloader, device):
    # Compute the Inception Score for the current fold for WGAN
    preds_fake = []
    for fake_batch in test_dataloader:
        fake_batch = fake_batch.to(device)

        # Resize generated images to match InceptionV3 input size
        fake_batch = nn.functional.interpolate(repeat_channel(fake_batch), size=(299, 299), mode='bilinear', align_corners=False)

        # Calculate predictions for generated images
        fake_preds = inception_model(fake_batch)
        preds_fake.append(softmax(fake_preds, dim=1))


    preds_fake = torch.cat(preds_fake, dim=0)

    # Compute the marginal entropy and the conditional entropy for generated samples
    marginal_entropy_fake = torch.sum(preds_fake * torch.log(preds_fake), dim=1)
    conditional_entropy_fake = torch.sum(preds_fake.mean(dim=0) * torch.log(preds_fake.mean(dim=0)))

    # Compute the Inception Score for WGAN
    inception_score_fake = torch.exp(conditional_entropy_fake - marginal_entropy_fake.mean())

    return inception_score_fake.item()



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
inception_scores_fake = calculate_inception_score_cv_wgan(generator, get_noise, num_samples=100, k_folds=5, device=device)
print("Inception Scores for Generated Samples (each fold):", inception_scores_fake)

Inception Scores for Generated Samples (each fold): [0.6659111380577087, 0.7126535177230835, 0.6684061884880066, 0.718695878982544, 0.6856997013092041]


In [39]:
np.mean([0.6659111380577087, 0.7126535177230835, 0.6684061884880066, 0.718695878982544, 0.6856997013092041])

0.6902732849121094

In [ ]:
b

NameError: ignored